In [3]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import sklearn

from delayedsparse import delayedsparse

In [8]:
# sparse matrices
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix

In [7]:
# load data
path = '/home/livtollanes/NewData/edgelist.csv'

req_cols = ['marker_id', 'follower_id', 'twitter_name']
dtypes = {'marker_id': 'object',
          'follower_id': 'object',
          'twitter_name': 'object'}


edgelist_CA = pd.read_csv(path, usecols=req_cols, dtype=dtypes)

In [ ]:
# sparse CA


Number_of_dimensions = 15




markers = bipartite_df['twitter_id'].unique()
followers = bipartite_df['followers_id'].unique()

Number_of_markers = markers.size
Number_of_followers = followers.size


markers_dic = pd.Series(index=bipartite_df['twitter_id'].unique(),data=np.arange(Number_of_markers,dtype=int)) # these are column positions of markers
followers_dic = pd.Series(index=bipartite_df['followers_id'].unique(),data=np.arange(Number_of_followers,dtype=int)) # these are column positions of followers


# merging row and col matrix indices into a new columgn in the bipartite graph dataframe
bipartite_df['i'] = bipartite_df['twitter_id'].map(markers_dic)
bipartite_df['j'] = bipartite_df['followers_id'].map(followers_dic)


# dropgin repeated edges just in case
bipartite_df.dropna(inplace=True)

# creating the COO matrix
coo = coo_matrix((np.ones((bipartite_df.shape[0])), (bipartite_df['j'].values, bipartite_df['i'])), shape=(Number_of_followers, Number_of_markers))
X = coo.tocsr()

# computing the sparse CA
ca = delayedsparse.CA(n_components=Number_of_dimensions)
ca.fit(X)


# extracting results
aug_inter = np.array(ca.explained_inertia_)


dimensions_names = ['latent_dim_%d'%(d+1) for d in range(Number_of_dimensions)]


scaler = StandardScaler(with_std=True)
scaler.fit(np.vstack(((ca.F*1)[:,:MPs_dims.shape[0]],(ca.G*1)[:,:MPs_dims.shape[0]])))    

followers_coords = pd.DataFrame(index=followers,columns=dimensions_names,data=scaler.transform((ca.F*1)[:,:Number_of_dimensions]))
markers_coords = pd.DataFrame(index=markers,columns=dimensions_names,data=scaler.transform((ca.G*1)[:,:Number_of_dimensions]))